In [6]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# Parameters for Indeed scraper: 
jobTitle = 'Web Developer'
location = 'San Francisco'
maxJobs = 2000 # Can only grab jobs in increments of 10 for now.
# Going to remove salaryEstimate for now because so many jobs don't list a salary. 
# We can filter by salary after the data is collected.
# salaryEstimate = "50" # This number represents thousands of dollars. i.e. 50 = $50,000

# Target URL Format: https://www.indeed.com/jobs?q=Data+Scientist&l=New+York

def makeURLs(jobTitle, location, maxJobs):
    
    numPages = maxJobs / 10
    numPages = int(numPages)
    
    root = 'http://www.indeed.com/jobs?q='
    
    jobTitle = jobTitle.replace(' ', '+')
    location = location.replace(' ', '+')
    
    URLs = []
    
    pageNum = 0
    for page in range(numPages):
    
        if pageNum == 0:
            seedURL = root + jobTitle + '&l=' + location
        else:
            jobNum = pageNum*10
            jobNum = str(jobNum)
            seedURL = root + jobTitle + '&l=' + location + '&start=' + jobNum
        pageNum = pageNum + 1
        URLs.append(seedURL)
        
    return URLs

seedURLs = makeURLs(jobTitle, location, maxJobs)

In [3]:
#Get URL for every job listed on the page. We'll remove duplicates later.

# Get list of links to Listing
def extract_listing_url_from_seed(URLs): 
    
    listingURLs = []
    
    for seedURL in URLs:
        print(seedURL)
         
        # Get HTML for each Seed URL to extract links to job listing pages.
        page = requests.get(seedURL)
        time.sleep(1) # One second pause between requests.
        soup = BeautifulSoup(page.text, 'html.parser')
        
        # Loop through html and append to list all page URLs linked to by job title.
        for div in soup.find_all(name='div', attrs={'class':'row'}):
            for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
                #print(a['href'])
                listingURLs.append(a['href'])
            
    #Remove URLs containing the string '/pagead/' or '/company'      
    listingURLs = [ x for x in listingURLs if '/pagead/' not in x ]
    listingURLs = [ x for x in listingURLs if '/company/' not in x ]
    
    return listingURLs

finalURLs = extract_listing_url_from_seed(seedURLs)

http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=10
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=20
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=30
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=40
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=50
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=60
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=70
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=80
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=90
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=100
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=110
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=120
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=130
http://www.indeed.com/jobs?q=Web+Developer&l=San+Fran

http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1190
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1200
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1210
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1220
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1230
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1240
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1250
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1260
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1270
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1280
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1290
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1300
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1310
http://www.indeed.com/jobs?q=Web+Developer&l=San+Francisco&start=1320
http://www.indeed.co

In [4]:
print(len(finalURLs))
print(finalURLs)

1902
['/rc/clk?jk=62b685cae257fb87&fccid=af586ce573e5d5ae&vjs=3', '/rc/clk?jk=f673cf9656d19eb6&fccid=318a4042e5b1e9cf&vjs=3', '/rc/clk?jk=c826035770b3b6b3&fccid=c6b89555ff30e032&vjs=3', '/rc/clk?jk=c18fca8622454c6d&fccid=eb507df48b2f81c7&vjs=3', '/rc/clk?jk=6ce4d45d6c9515a2&fccid=0a5b3a6b433d042b&vjs=3', '/rc/clk?jk=9efd487002a60550&fccid=734635fac83ee605&vjs=3', '/rc/clk?jk=f67e5e98ab7546bf&fccid=f89deb5a97c7738a&vjs=3', '/rc/clk?jk=d847d879917a99d5&fccid=41e8f9abd9840a7a&vjs=3', '/rc/clk?jk=f1998119d1e3df1a&fccid=c0be56e2ea014be0&vjs=3', '/rc/clk?jk=acc372d147d5caf2&fccid=431a376f2ec40d37&vjs=3', '/rc/clk?jk=9eba08403338dc1c&fccid=ecf539b4f66ba031&vjs=3', '/rc/clk?jk=c33198c682a0ea60&fccid=50b0278527d9da6e&vjs=3', '/rc/clk?jk=67764d9c0c986df9&fccid=8510f9bc3a085f8e&vjs=3', '/rc/clk?jk=b9b29482c9f953c2&fccid=dd616958bd9ddc12&vjs=3', '/rc/clk?jk=303577c7173b63e0&fccid=1f773401da480d04&vjs=3', '/rc/clk?jk=d8e240b8eda46ef9&fccid=3416a04caf5bafd0&vjs=3', '/rc/clk?jk=4d99c63c50f685de&fccid

In [5]:
job_titles = []
    
companies = []

locations = []

summaries = []

for url in finalURLs:
    
    # Get HTML of the Page
    URL = 'http://www.indeed.com/' + url
    page = requests.get(URL)
    time.sleep(1)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Get Job Title
    for b in soup.find_all(name='b', attrs={'class': 'jobtitle'}):
        job_titles.append(b.text)
    
    # Get Company Name
    for div in soup.find_all(name='div', attrs={'data-tn-component':'jobHeader'}):
        for span in div.find_all(name='span', attrs={'class': 'company'}):
            companies.append(span.text)
            
    # Get Location
    for div in soup.find_all(name='div', attrs={'data-tn-component':'jobHeader'}):
        for span in div.find_all(name='span', attrs={'class': 'location'}):
            locations.append(span.text)

    # Get Summary
    for span in soup.find_all(name='span', attrs={'class': 'summary'}):
        summaries.append(span.text)   
            
# Lists -> Series to prepare for DataFrame
job_titles = pd.Series(job_titles)
companies = pd.Series(companies)
locations = pd.Series(locations)
summaries = pd.Series(summaries)

# Create new DataFrame
df = pd.DataFrame()

# Assign column titles and set values
df['job_title'] = job_titles.values
df['company'] = companies.values
df['location'] = locations.values
df['summary'] = summaries.values

# Export to CSV
df.to_csv('~/Desktop/output.csv')